In [31]:
from Tradables import Underlier
import pickle
import gs_quant
from gs_quant.session import GsSession
from datetime import datetime, timedelta
import investpy
from gs_quant.analytics.datagrid import DataColumn
from gs_quant.analytics.processors import LastProcessor, EntityProcessor
from gs_quant.data.coordinate import DataCoordinate, DataMeasure, DataFrequency
from gs_quant.analytics.datagrid import DataRow
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.analytics.datagrid import DataColumn, DataRow, DimensionsOverride, DataGrid
from gs_quant.analytics.processors import LastProcessor, EntityProcessor, CoordinateProcessor
from gs_quant.data import DataDimension
from gs_quant.data.coordinate import DataCoordinate, DataMeasure, DataFrequency
from gs_quant.markets.securities import AssetIdentifier, Asset
from gs_quant.session import GsSession
import pandas as pd
GsSession.use(
    client_id="3e871897e5474a0488cb09ea699ffe1c",
    client_secret="3dd2854652249d480cb7e8724ff23ee892dde030da367d24917565183b2ae929"
)
date_offsets_fwd = {
    'Overnight': timedelta(days=1),
    'Tomorrow Next': timedelta(days=1),
    'Spot Next': timedelta(days=2),
    'One Week': timedelta(weeks=1),
    'Two Weeks': timedelta(weeks=2),
    'Three Weeks': timedelta(weeks=3),
    'One Month': timedelta(days=30),
    'Two Months': timedelta(days=60),
    'Three Months': timedelta(days=90),
    'Four Months': timedelta(days=120),
    'Five Months': timedelta(days=150),
    'Six Months': timedelta(days=180),
    'Seven Months': timedelta(days=210),
    'Eight Months': timedelta(days=240),
    'Nine Months': timedelta(days=270),
    'Ten Months': timedelta(days=300),
    'Eleven Months': timedelta(days=330),
    'One Year': timedelta(days=365),
    'Two Years': timedelta(days=730),
    'Three Years': timedelta(days=1095),
    'Four Years': timedelta(days=1460),
    'Five Years': timedelta(days=1825),
    'Six Years': timedelta(days=2190),
    'Seven Years': timedelta(days=2555),
    'Ten Years': timedelta(days=3650),
}

date_offsets_vol = {
    '1D': timedelta(days=1),
    '1W': timedelta(weeks=1),
    '2W': timedelta(weeks=2),
    '3W': timedelta(weeks=3),
    '1M': timedelta(days=30),
    '2M': timedelta(days=60),
    '3M': timedelta(days=90),
    '6M': timedelta(days=180),
    '9M': timedelta(days=270),
    '1Y': timedelta(days=365)
}

In [32]:
eurusd = Underlier("EURUSD")
eurusd.mark_spot(1.07)
eurusd.mark_vol(.095)
def save_security(sec):
    """save security to pickle in the securities folder"""
    with open(f"Securities/{sec.name}.pkl", "wb") as f:
        pickle.dump(sec, f)
save_security(eurusd)


In [33]:
"""use excel wings to load surface from EURUSD Risk&Marks"""
import os
guipath = os.path.abspath('C:\\Users\\jacob\\bolt-hub\\GUI.xlsx')
marks = pd.read_excel(guipath, sheet_name="EURUSD Risks&Marks")


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Forward Curve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Expiration,Ask,Bid,Mid,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Overnight,1.07637,1.07635,1.07636,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tomorrow Next,1.07637,1.07635,1.07636,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Spot Next,1.07638,1.07635,1.07636,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,One Week,1.07675,1.07671,1.07673,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
def extract_forward_curve(marks):
    """extract forward curve"""
    fwd = marks.drop(index=range(5))
    last_row_index = fwd.index[-31]
    # Select all rows up to and including the last 31st row, and assign it back to the dataframe
    fwd = fwd.iloc[:last_row_index+1, :]
    fwd = fwd.iloc[:, :4]
    fwd.iloc[1:, 0] = [date_offsets_fwd[tenor] for tenor in fwd.iloc[1:, 0]]
    # get the values of the top row as a series
    new_columns = fwd.iloc[0]
    # assign the new column names to the dataframe
    fwd.columns = new_columns
    # drop the first row since it's no longer needed as column names
    fwd = fwd.iloc[1:]
    # set the index to the values in the first column by position
    fwd = fwd.set_index(fwd.columns[0])
    # drop the first column since it's now the index
    fwd = fwd.drop(columns=[fwd.columns[0]])
    fwd = fwd.rename_axis(index=None).rename_axis(columns=None)
    fwd.drop('Bid', axis=1, inplace=True)
    return fwd

def extract_vol_surface(marks):
    """extract vol surface"""
    vols = marks.drop(index=range(33))
    vols = vols.iloc[0:, :18]
    # get the values of the top row as a series
    new_columns = vols.iloc[0]
    # assign the new column names to the dataframe
    vols.columns = new_columns
    # drop the first row since it's no longer needed as column names
    vols = vols.iloc[1:]
    # set the index to the values in the first column by position
    vols = vols.set_index(vols.columns[0])
    # drop the first column since it's now the index
    vols = vols.drop(columns=[vols.columns[0]])
    vols = vols.rename_axis(index=None).rename_axis(columns=None)
    return vols

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
34,1 days,9.54,9.4,9.32,9.27,9.22,9.18,9.15,9.13,9.08,...,9,8.98,8.97,8.97,8.98,1D,-0.23,-0.43,0.03,0.11
35,7 days,8.13,8.07,8.03,8,7.98,7.95,7.93,7.92,7.88,...,7.8,7.78,7.76,7.73,7.69,1W,-0.18,-0.34,0,0.02
36,14 days,7.29,7.17,7.1,7.06,7.02,6.99,6.97,6.95,6.92,...,6.88,6.88,6.89,6.9,6.94,2W,-0.14,-0.27,0.03,0.12
37,21 days,7.21,6.99,6.87,6.79,6.73,6.68,6.65,6.62,6.57,...,6.55,6.57,6.59,6.64,6.75,3W,-0.18,-0.35,0.07,0.25
38,30 days,7.29,7.02,6.88,6.78,6.71,6.66,6.61,6.57,6.52,...,6.49,6.51,6.54,6.6,6.72,1M,-0.22,-0.43,0.08,0.29
39,60 days,7.74,7.31,7.08,6.92,6.81,6.72,6.65,6.59,6.5,...,6.41,6.43,6.46,6.52,6.67,2M,-0.39,-0.79,0.11,0.41
40,90 days,8.04,7.45,7.16,6.96,6.82,6.71,6.62,6.55,6.44,...,6.35,6.36,6.4,6.49,6.68,3M,-0.47,-0.96,0.14,0.53
41,180 days,9.14,8.1,7.66,7.38,7.18,7.03,6.91,6.81,6.67,...,6.53,6.54,6.59,6.7,6.94,6M,-0.66,-1.4,0.19,0.73
42,270 days,10.61,8.71,8.1,7.74,7.49,7.3,7.15,7.02,6.84,...,6.66,6.67,6.72,6.84,7.11,9M,-0.83,-1.88,0.23,0.93
43,365 days,18.57,9.24,8.45,8.01,7.71,7.49,7.31,7.17,6.97,...,6.75,6.77,6.82,6.95,7.24,1Y,-0.96,-2.29,0.26,1.13
